In [27]:
import pandas as pd
import datetime as dtime
from datetime import datetime
from functions import analysis, feature_engineering, train_test_split, create_model, cv, baseline, \
    test_model, features, test_baseline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
data = pd.read_csv("data.csv")
# выкинем колонки, которые не нужы для 1 го задания
data = data.drop(columns=['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 
                  'g0', 'g1', 'coeff_sum0', 'coeff_sum1'])
data.head()

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,0,0,0,1,1
1,2021-09-26 22:54:49.000000,1,1,0,0,1,1,1
2,2021-09-26 23:57:20.000000,2,2,3,0,0,1,1
3,2021-09-27 00:04:30.000000,3,3,0,1,1,1,1
4,2021-09-27 00:06:21.000000,4,4,0,1,0,1,1


In [29]:
analysis(data)

Количество записей в таблице:  15821472
соотношние между классами:
1-ый класс: 2.6688351121817235 %, 2-ой класс 97.33116488781828 %

Nan в таблице:
False

Количество уникальных значений для os_id:  11
Количество уникальных значений для country_id:  17
Количество уникальных значений для impressions:  1
Количество уникальных значений для campaign_clicks:  822
Количество уникальных значений для banner_id:  1633


In [30]:
# Nan нет в таблице,поэтому пропуски заполнять не надо
# Так как значение этой колоники одинаково для всех строк, уберем ее
data = data.drop(columns=['impressions'])

#### Проанализируем чем отличаются случаи, когда были клики, от остальных

Посмотрим распределение по странам, операционным системам в случаях, когда был клик, и когда не было (у других фичей слишком много значений)

In [31]:
# по странам
countries_dist_click = pd.DataFrame(data.loc[data['clicks']==1]['country_id'].value_counts(normalize=True)).reset_index()
countries_dist_no_click = pd.DataFrame(data.loc[data['clicks']==0]['country_id'].value_counts(normalize=True)).reset_index()
countries_dist_no_click.merge(countries_dist_click, 'inner', 'index').rename({'country_id_x':'no_click_countries',
                                                                              'country_id_y':'click_countries' }, axis=1)
# По результатам не видно корреляцией между старной и обилием кликов за редкими исключениями вроде country_id=16


,index,no_click_countries,click_countries
0,0,0.313052,0.321239
1,5,0.121233,0.102226
2,6,0.101050,0.063022
3,7,0.096413,0.096815
4,3,0.084440,0.087806
5,1,0.084344,0.112474
6,15,0.038368,0.029520
7,12,0.030311,0.026698
8,9,0.027631,0.031981
9,4,0.024601,0.023590


In [32]:
# по операционным системам
os_dist_click = pd.DataFrame(data.loc[data['clicks']==1]['os_id'].value_counts(normalize=True)).reset_index()
os_dist_no_click = pd.DataFrame(data.loc[data['clicks']==0]['os_id'].value_counts(normalize=True)).reset_index()
os_dist_no_click.merge(os_dist_click, 'inner', 'index').rename({'os_id_x':'no_click_os',
                                                                              'os_id_y':'click_os' }, axis=1)
# Эта фича куда более говорящая - на некоторых операционных системах люди кликают рекламу реже (os_id=2, 1), на других - чаще (os_id=3, 4, 5, 6)

,index,no_click_os,click_os
0,2,0.292751,0.193838
1,0,0.242925,0.274584
2,1,0.203629,0.101767
3,4,0.187886,0.282404
4,3,0.046968,0.086447
5,6,0.018872,0.046747
6,5,0.006882,0.014131
7,8,0.000081,0.000073
8,7,0.000006,0.000007


# Построение модели

In [ ]:
# Добавим признаков
data = feature_engineering(data)
data.tail()

In [6]:
# Разобъем на тест и тест: в качестве теста возьмем последний день
X_train, Y_train, X_test, Y_test = train_test_split(data)

In [7]:
model = create_model(X_train, Y_train)
test_model(model,X_test, Y_test )

C:\Users\msk-lokteva\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\msk-lokteva\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_ranking.py:1018: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


Log loss: 0.16119478661755937
Auc: nan


In [29]:
test_model(model,X_test, Y_test )

[[0.98564307 0.01435693]
 [0.97183289 0.02816711]
 [0.98421781 0.01578219]
 ...
 [0.98746261 0.01253739]
 [0.98966826 0.01033174]
 [0.98715306 0.01284694]]
[0.01435693 0.02816711 0.01578219 ... 0.01253739 0.01033174 0.01284694]


C:\Users\msk-lokteva\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_ranking.py:1018: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


Log loss: 0.15994777693560325
Auc: nan


In [14]:
Y_train.value_counts(normalize=True)

0    0.974666
1    0.025334
Name: clicks, dtype: float64

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,clicks,time,date,time_of_day,is_weekend,month,15_av_clicks_os,15_av_clicks_country,15_av_clicks_month
0,2021-09-27 00:01:30,0,0,0,0,0,1,00:01:30,2021-09-27,3,0,9,-1.0,-1.000000,-1.0
1,2021-09-26 22:54:49,1,1,0,0,1,1,22:54:49,2021-09-26,2,1,9,-1.0,-1.000000,-1.0
2,2021-09-26 23:57:20,2,2,3,0,0,1,23:57:20,2021-09-26,2,1,9,-1.0,-1.000000,-1.0
3,2021-09-27 00:04:30,3,3,0,1,1,1,00:04:30,2021-09-27,3,0,9,-1.0,-1.000000,-1.0
4,2021-09-27 00:06:21,4,4,0,1,0,1,00:06:21,2021-09-27,3,0,9,-1.0,-1.000000,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239371,2021-10-02 23:30:21,14,48,2,4,1,1,23:30:21,2021-10-02,2,1,10,0.0,0.066667,0.0
2239372,2021-10-02 23:49:36,1,220,0,4,15,1,23:49:36,2021-10-02,2,1,10,0.0,0.066667,0.0
2239373,2021-10-02 23:50:27,19,594,0,0,3,1,23:50:27,2021-10-02,2,1,10,0.0,0.066667,0.0
2239374,2021-10-02 23:57:30,1,0,0,0,0,1,23:57:30,2021-10-02,2,1,10,0.0,0.066667,0.0


In [7]:
# Подберем параметры регуляризации:
l1, alpha = cv(X_train, Y_train, 'roc_auc')

C:\Users\msk-lokteva\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.686e+05, tolerance: 3.381e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.2, L1: 0.0, score: [0.5 nan nan]


C:\Users\msk-lokteva\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.687e+05, tolerance: 3.381e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.4666666666666667, L1: 0.0, score: [0.5 nan nan]


C:\Users\msk-lokteva\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.687e+05, tolerance: 3.381e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Alpha: 0.7333333333333334, L1: 0.0, score: [0.5 nan nan]


C:\Users\msk-lokteva\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.688e+05, tolerance: 3.381e+01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


Alpha: 1.0, L1: 0.0, score: [0.5 nan nan]
Alpha: 0.2, L1: 0.25, score: [0.5 nan nan]
Alpha: 0.4666666666666667, L1: 0.25, score: [0.5 nan nan]
Alpha: 0.7333333333333334, L1: 0.25, score: [0.5 nan nan]
Alpha: 1.0, L1: 0.25, score: [0.5 nan nan]
Alpha: 0.2, L1: 0.5, score: [0.5 nan nan]
Alpha: 0.4666666666666667, L1: 0.5, score: [0.5 nan nan]
Alpha: 0.7333333333333334, L1: 0.5, score: [0.5 nan nan]
Alpha: 1.0, L1: 0.5, score: [0.5 nan nan]
Alpha: 0.2, L1: 0.75, score: [0.5 nan nan]
Alpha: 0.4666666666666667, L1: 0.75, score: [0.5 nan nan]
Alpha: 0.7333333333333334, L1: 0.75, score: [0.5 nan nan]
Alpha: 1.0, L1: 0.75, score: [0.5 nan nan]
Alpha: 0.2, L1: 1.0, score: [0.5 nan nan]
Alpha: 0.4666666666666667, L1: 1.0, score: [0.5 nan nan]
Alpha: 0.7333333333333334, L1: 1.0, score: [0.5 nan nan]
Alpha: 1.0, L1: 1.0, score: [0.5 nan nan]


In [8]:
print(l1, alpha)

0.0 0.2


In [10]:

tuned_model = create_model(X_train, Y_train, l1, alpha)


KeyboardInterrupt: 

In [ ]:
test_model(tuned_model, X_test, Y_test)

In [51]:
import sklearn

sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_

In [40]:
# Сравним с бейзлайном
data = baseline(data)
test_baseline(data)

C:\Users\msk-lokteva\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_ranking.py:1018: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


Log loss: 0.11234095165091723
Auc: nan


In [ ]:
# Подсчет среднего в бейзлайне дал околонулевые верятности кликов - поэтому Auc считать бессмысленно